In [1]:
'''
Johnson Controls Metasys archive cleaning utility. 
Reads .xml file from a Metasys SCT export file (.zip), 
parses objects under the integration, and allows the
user to delete object using regular expressions.

Developed 2018-08-02 by Chris Weyandt, Lawrence Berkeley National lab
'''

'\nJohnson Controls Metasys archive cleaning utility. \nReads .xml file from a Metasys SCT export file (.zip), \nparses objects under the integration, and allows the\nuser to delete object using regular expressions.\n\nDeveloped 2018-08-02 by Chris Weyandt, Lawrence Berkeley National lab\n'

In [79]:
from bs4 import BeautifulSoup
from bs4.element import NavigableString
import re

In [16]:
xml_sample = """
<objects xmlns="http://johnsoncontrols.com/MetasysIII/2002/3/Core" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <object classid="176" classVersion="1.0" ref="metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_259" objectid="3009791">
    <property id="28">
      <data>
        <string>CLOSET (CRASH RESET)</string>
      </data>
    </property>
    <property id="75">
      <data>
        <BACoid id="176">3009791</BACoid>
      </data>
    </property>
    <property id="2390">
      <data>
        <string>02_259</string>
      </data>
    </property>
    <property id="32527">
      <data>
        <string>metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_259</string>
      </data>
    </property>
  </object>
  <object classid="176" classVersion="1.0" ref="metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_260A" objectid="3009798">
    <property id="28">
      <data>
        <string>WET LAB</string>
      </data>
    </property>
    <property id="75">
      <data>
        <BACoid id="176">3009798</BACoid>
      </data>
    </property>
    <property id="2390">
      <data>
        <string>02_260A</string>
      </data>
    </property>
    <property id="32527">
      <data>
        <string>metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_260A</string>
      </data>
    </property>
  </object>
  <object classid="176" classVersion="1.0" ref="metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_260B" objectid="3009810">
    <property id="28">
      <data>
        <string>WET LAB</string>
      </data>
    </property>
    <property id="75">
      <data>
        <BACoid id="176">3009810</BACoid>
      </data>
    </property>
    <property id="2390">
      <data>
        <string>02_260B</string>
      </data>
    </property>
    <property id="32527">
      <data>
        <string>metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_260B</string>
      </data>
    </property>
  </object>
  <object classid="176" classVersion="1.0" ref="metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_261A" objectid="3009822">
    <property id="28">
      <data>
        <string>WET LAB</string>
      </data>
    </property>
    <property id="75">
      <data>
        <BACoid id="176">3009822</BACoid>
      </data>
    </property>
    <property id="2390">
      <data>
        <string>02_261A</string>
      </data>
    </property>
    <property id="32527">
      <data>
        <string>metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_261A</string>
      </data>
    </property>
  </object>
  <object classid="176" classVersion="1.0" ref="metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_261C" objectid="3009845">
    <property id="28">
      <data>
        <string>WET LAB</string>
      </data>
    </property>
    <property id="75">
      <data>
        <BACoid id="176">3009845</BACoid>
      </data>
    </property>
    <property id="2390">
      <data>
        <string>02_261C</string>
      </data>
    </property>
    <property id="32527">
      <data>
        <string>metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_261C</string>
      </data>
    </property>
  </object>
  <object classid="176" classVersion="1.0" ref="metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_263A" objectid="3009861">
    <property id="28">
      <data>
        <string>WET LAB</string>
      </data>
    </property>
    <property id="75">
      <data>
        <BACoid id="176">3009861</BACoid>
      </data>
    </property>
    <property id="2390">
      <data>
        <string>02_263A</string>
      </data>
    </property>
    <property id="32527">
      <data>
        <string>metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_263A</string>
      </data>
    </property>
  </object>
  <object classid="176" classVersion="1.0" ref="metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_263B" objectid="3009871">
    <property id="28">
      <data>
        <string>WET LAB(CRASH BUTTON)</string>
      </data>
    </property>
    <property id="75">
      <data>
        <BACoid id="176">3009871</BACoid>
      </data>
    </property>
    <property id="2390">
      <data>
        <string>02_263B</string>
      </data>
    </property>
    <property id="32527">
      <data>
        <string>metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_263B</string>
      </data>
    </property>
  </object>
  <object classid="176" classVersion="1.0" ref="metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_MAP2" objectid="3009321">
    <property id="28">
      <data>
        <string>NC2_2 MAPPED POINTS</string>
      </data>
    </property>
    <property id="75">
      <data>
        <BACoid id="176">3009321</BACoid>
      </data>
    </property>
    <property id="2390">
      <data>
        <string>02_MAP2</string>
      </data>
    </property>
    <property id="32527">
      <data>
        <string>metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_MAP2</string>
      </data>
    </property>
  </object>
  <object classid="176" classVersion="1.0" ref="metasys:Metasys-nie1/LBNL_FMS.NC2_3.02_LBY3" objectid="3009931">
    <property id="28">
      <data>
        <string>LOBBY (R300 WEST)</string>
      </data>
    </property>
    <property id="75">
      <data>
        <BACoid id="176">3009931</BACoid>
      </data>
    </property>
    <property id="2390">
      <data>
        <string>02_LBY3</string>
      </data>
    </property>
    <property id="32527">
      <data>
        <string>metasys:Metasys-nie1/LBNL_FMS.NC2_3.02_LBY3</string>
      </data>
    </property>
  </object>
  <object classid="156" classVersion="1.0" ref="metasys:Metasys-nie1/LBNL_FMS.NC2_1.02_LBY1.SCH-OCC.ObjAlarm" objectid="3024404">
    <property id="75">
      <data>
        <BACoid id="156">3024404</BACoid>
      </data>
    </property>
    <property id="132">
      <data>
        <attrref id="85" itemName="02_LBY1.SCH-OCC">metasys:Metasys-nie1/LBNL_FMS.NC2_1.02_LBY1.SCH-OCC</attrref>
      </data>
    </property>
    <property id="530">
      <data>
        <boolean>0</boolean>
      </data>
    </property>
    <property id="627">
      <data>
        <float>2.000000</float>
      </data>
    </property>
    <property id="2390">
      <data>
        <string>ObjAlarm</string>
      </data>
    </property>
    <property id="32527">
      <data>
        <string>metasys:Metasys-nie1/LBNL_FMS.NC2_1.02_LBY1.SCH-OCC.ObjAlarm</string>
      </data>
    </property>
  </object>
</objects>
"""

In [81]:
file = io.open('export.xml')

In [124]:
soup = BeautifulSoup(xml_sample, features="xml")

In [65]:
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<objects xmlns="http://johnsoncontrols.com/MetasysIII/2002/3/Core" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
 <object classVersion="1.0" classid="176" objectid="3009791" ref="metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_259">
  <property id="28">
   <data>
    <string>
     CLOSET (CRASH RESET)
    </string>
   </data>
  </property>
  <property id="75">
   <data>
    <BACoid id="176">
     3009791
    </BACoid>
   </data>
  </property>
  <property id="2390">
   <data>
    <string>
     02_259
    </string>
   </data>
  </property>
  <property id="32527">
   <data>
    <string>
     metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_259
    </string>
   </data>
  </property>
 </object>
 <object classVersion="1.0" classid="176" objectid="3009798" ref="metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_260A">
  <property id="28">
   <data>
    <string>
     WET LAB
    </string>
   </data>
  </property>
  <property id="75">
   <data>
    <BACoid id="176">
     300979

In [66]:
for child in soup.objects:
    print(child)



<object classVersion="1.0" classid="176" objectid="3009791" ref="metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_259">
<property id="28">
<data>
<string>CLOSET (CRASH RESET)</string>
</data>
</property>
<property id="75">
<data>
<BACoid id="176">3009791</BACoid>
</data>
</property>
<property id="2390">
<data>
<string>02_259</string>
</data>
</property>
<property id="32527">
<data>
<string>metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_259</string>
</data>
</property>
</object>


<object classVersion="1.0" classid="176" objectid="3009798" ref="metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_260A">
<property id="28">
<data>
<string>WET LAB</string>
</data>
</property>
<property id="75">
<data>
<BACoid id="176">3009798</BACoid>
</data>
</property>
<property id="2390">
<data>
<string>02_260A</string>
</data>
</property>
<property id="32527">
<data>
<string>metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_260A</string>
</data>
</property>
</object>


<object classVersion="1.0" classid="176" objectid="3009810" ref="metasys:M

In [67]:
print(soup.children)

In [68]:
type(soup.objects)

bs4.element.Tag

In [70]:
soup.objects.object.findAll("id")

[]

In [71]:
for x in soup.objects.children:
    if type(x) == NavigableString:
        continue
    #print( type(x) )
    for z in x.children:
        print(z)
    print()
    tag = x
    



<property id="28">
<data>
<string>CLOSET (CRASH RESET)</string>
</data>
</property>


<property id="75">
<data>
<BACoid id="176">3009791</BACoid>
</data>
</property>


<property id="2390">
<data>
<string>02_259</string>
</data>
</property>


<property id="32527">
<data>
<string>metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_259</string>
</data>
</property>





<property id="28">
<data>
<string>WET LAB</string>
</data>
</property>


<property id="75">
<data>
<BACoid id="176">3009798</BACoid>
</data>
</property>


<property id="2390">
<data>
<string>02_260A</string>
</data>
</property>


<property id="32527">
<data>
<string>metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_260A</string>
</data>
</property>





<property id="28">
<data>
<string>WET LAB</string>
</data>
</property>


<property id="75">
<data>
<BACoid id="176">3009810</BACoid>
</data>
</property>


<property id="2390">
<data>
<string>02_260B</string>
</data>
</property>


<property id="32527">
<data>
<string>metasys:Metasys-nie1/LBNL_FMS.

In [106]:
#Get Tag Attributes#
def get_tag_attr( tag, ttype=list):
    attr_flag = False
    current_tag = {}
    if hasattr( tag, 'attrs'):
        if len(tag.attrs) > 0:
            attr_flag = True
            if ttype == list:
                current_tag.update(tag.attrs)
            else:
                current_tag[tag.name].update(tag.attrs)
    return current_tag
   # print(current_tag['ref'])

In [82]:
bigSoup = BeautifulSoup(file, "xml")

In [83]:
i = 0
for x in bigSoup.objects.children:
    if type(x) == NavigableString:
        continue
    #print( type(x) )
    #for z in x.children:
    #    print(z)
    #print()
    #tag = x
    i += 1
print(i)

38376


In [95]:
print( enumerate(soup.objects.children) )

In [109]:
for i, val in enumerate(soup.objects.children):    #convert list_itter object to enumberated list, allowing to call by index.
    if type(val) == NavigableString:
        continue
    print(i, type(val), get_tag_attr(val)['ref'])

1 <class 'bs4.element.Tag'> metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_259
3 <class 'bs4.element.Tag'> metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_260A
5 <class 'bs4.element.Tag'> metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_260B
7 <class 'bs4.element.Tag'> metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_261A
9 <class 'bs4.element.Tag'> metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_261C
11 <class 'bs4.element.Tag'> metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_263A
13 <class 'bs4.element.Tag'> metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_263B
15 <class 'bs4.element.Tag'> metasys:Metasys-nie1/LBNL_FMS.NC2_2.02_MAP2
17 <class 'bs4.element.Tag'> metasys:Metasys-nie1/LBNL_FMS.NC2_3.02_LBY3
19 <class 'bs4.element.Tag'> metasys:Metasys-nie1/LBNL_FMS.NC2_1.02_LBY1.SCH-OCC.ObjAlarm


In [122]:
newSoup = BeautifulSoup("", "xml")

In [127]:
for val in soup.objects.children:    
    if type(val) == NavigableString:
        continue    
    #print(get_tag_attr(val)['ref'])
    if val.find("property") is not None:
        print(val.find("property"))
    #val.decompose() #removes element

<property id="28">
<data>
<string>CLOSET (CRASH RESET)</string>
</data>
</property>
<property id="28">
<data>
<string>WET LAB</string>
</data>
</property>
<property id="28">
<data>
<string>WET LAB</string>
</data>
</property>
<property id="28">
<data>
<string>WET LAB</string>
</data>
</property>
<property id="28">
<data>
<string>WET LAB</string>
</data>
</property>
<property id="28">
<data>
<string>WET LAB</string>
</data>
</property>
<property id="28">
<data>
<string>WET LAB(CRASH BUTTON)</string>
</data>
</property>
<property id="28">
<data>
<string>NC2_2 MAPPED POINTS</string>
</data>
</property>
<property id="28">
<data>
<string>LOBBY (R300 WEST)</string>
</data>
</property>
<property id="75">
<data>
<BACoid id="156">3024404</BACoid>
</data>
</property>


In [118]:
?NavigableString

Init signature: NavigableString(value)
Docstring:     
str(object='') -> str
str(bytes_or_buffer[, encoding[, errors]]) -> str

Create a new string object from the given object. If encoding or
errors is specified, then the object must expose a data buffer
that will be decoded using the given encoding and error handler.
Otherwise, returns the result of object.__str__() (if defined)
or repr(object).
encoding defaults to sys.getdefaultencoding().
errors defaults to 'strict'.
File:           ~/anaconda3/lib/python3.6/site-packages/bs4/element.py
Type:           type


In [ ]:
if node.find(re.compile("^h[1-6]"), recursive=False) is not None:
    return node.parent.parent